In [2]:
# Problem 1

In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [36]:
!pip install contractions


In [5]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [6]:

# Load Dataset
data = pd.read_csv('spam.csv', encoding='latin-1')[['v1', 'v2']]
data.columns = ['label', 'message']
data['label'] = data['label'].map({'ham': 0, 'spam': 1})

In [7]:

stop_words = set(stopwords.words('english'))

In [8]:
def preprocess(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return filtered_tokens

data['tokens'] = data['message'].apply(preprocess)

In [24]:
# loading Word2Vec
w2v_path = 'GoogleNews-vectors-negative300.bin'  
w2v_model = KeyedVectors.load_word2vec_format(w2v_path, binary=True)

In [25]:
def avg_word2vec(tokens, model, vector_size=300):
    vectors = [model[word] for word in tokens if word in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(vector_size)

data['vector'] = data['tokens'].apply(lambda x: avg_word2vec(x, w2v_model))

In [26]:

X = np.stack(data['vector'].values)
y = data['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [27]:
y_pred = clf.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))

Test Accuracy: 0.9426008968609866


In [28]:

def predict_message_class(model, w2v_model, message):
    tokens = preprocess(message)
    vector = avg_word2vec(tokens, w2v_model).reshape(1, -1)
    prediction = model.predict(vector)[0]
    return 'spam' if prediction == 1 else 'ham'

In [29]:

example = "Free entry in 2 a weekly competition to win FA Cup tickets"
print("Prediction:", predict_message_class(clf, w2v_model, example))


Prediction: spam


In [30]:
# Problem 2

In [38]:
import pandas as pd
import numpy as np
import re
import string
import nltk
import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [39]:

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
df = pd.read_csv('Tweets.csv')  
df = df[['airline_sentiment', 'text']]

In [41]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [42]:
def clean_tweet(text):
    text = text.lower()
    text = contractions.fix(text)
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  
    text = re.sub(r'@\w+', '', text)  
    text = re.sub(r'#\w+', '', text)  
    text = text.translate(str.maketrans('', '', string.punctuation))  
    text = re.sub(r'[^\x00-\x7F]+', '', text)  
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

df['tokens'] = df['text'].apply(clean_tweet)

In [43]:

w2v_path = 'GoogleNews-vectors-negative300.bin'  
w2v_model = KeyedVectors.load_word2vec_format(w2v_path, binary=True)

In [44]:

def avg_word2vec(tokens, model, vector_size=300):
    vectors = [model[word] for word in tokens if word in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(vector_size)

df['vector'] = df['tokens'].apply(lambda x: avg_word2vec(x, w2v_model))


In [45]:


X = np.stack(df['vector'].values)
y = df['airline_sentiment'].map({'negative': 0, 'neutral': 1, 'positive': 2}).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [46]:

y_pred = clf.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))


Test Accuracy: 0.7715163934426229


In [47]:

def predict_tweet_sentiment(model, w2v_model, tweet):
    tokens = clean_tweet(tweet)
    vector = avg_word2vec(tokens, w2v_model).reshape(1, -1)
    pred = model.predict(vector)[0]
    sentiment_map = {0: 'negative', 1: 'neutral', 2: 'positive'}
    return sentiment_map[pred]



In [48]:

example_tweet = "I love how Delta handled my flight change today!"
print("Predicted Sentiment:", predict_tweet_sentiment(clf, w2v_model, example_tweet))


Predicted Sentiment: positive
